<code>from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train =  scaler.transform(X_train)
X_test = scaler.transform(X_test)/<code>

In [68]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import median_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.datasets import load_boston

## load the boston dataset
boston = load_boston()
X, y = boston['data'], boston['target']
features = boston['feature_names']

## split the data to a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## create a pipeline
pipe = Pipeline([("scaler", StandardScaler()),
                 ("featsel", SelectKBest(k=10)),
                 ("rf", RandomForestRegressor(n_estimators=20))])

## train on the training data
pipe.fit(X_train, y_train)

## evaluate the model with the test data
y_pred = pipe.predict(X_test)
print(r'R^2=%.2f, MAE=%.2f'%(r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred))) 

R^2=0.75, MAE=1.69


<img src="images/scorespng.png "
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

# Dealing with unbalanced data
Conventionally the rare or minority class (e.g. fraud) is designated as the positive class, so two of the more commonly used alternative metrics to accuracy—precision and recall—have the true positives in their numerators. You’ll also see F1 score, which is the harmonic mean of precision and recall.

# Sampling Techniques
The most common approaches are sampling based, or more specifically, re-sampling based. Between up-sampling and down-sampling (also called over-sampling and under-sampling), down-sampling is a bit simpler conceptually. Fundamentally, if you have a minority class or classes that is noticeably underrepresented. In a random way you can drop some of those from the training data so that the proportions are more closely matched across classes. There are additional methods, one of which is inspired by K Nearest Neighbors (KNN) that can improve on pure random selection.

A major caveat to down-sampling is that we are not using all of our data. Over-sampling techniques also come in several flavors, from random or naive versions to classes of algorithms like the Synthetic Minority Oversampling Technique (SMOTE) [2] and the Adaptive Synthetic (ADASYN) [8] sampling method. Synthetic Minority Over-sampling Technique for Nominal and Continuous (SMOTENC) is an extension of the original SMOTE method designed to handle a mixture of categorical and continuous features. SMOTE has a number of other variants including ones that make use of Support Vector Machines and K-means clustering to improve on the synthetic samples.



# Class Imbalance Deep Dive
* Models sensitivity to class imbalance: some models are more or less sensitive to class imbalance
* Sampling techniques: up-sampling and down-sampling
* Imbalanced-learn: introduce the package and use SMOTE as an example

In [84]:
import pandas as pd
import os
from collections import Counter
from sklearn.metrics import classification_report

In [99]:
data_dir = os.path.join('.', 'data')
df = pd.read_csv(os.path.join(data_dir, r'aavail-target.csv'))
df.head()

customer_id   country_name   age     customer_name  is_subscriber  \
0            1  united_states  21.0        Kasen Todd           True   
1            2      singapore  31.0      Ensley Garza          False   
2            3  united_states  22.0     Lillian Carey          False   
3            4  united_states  21.0  Beau Christensen           True   
4            5      singapore  22.0    Ernesto Gibson           True   

    subscriber_type  num_streams  
0    aavail_premium           23  
1  aavail_unlimited           12  
2    aavail_premium           22  
3      aavail_basic           19  
4    aavail_premium           23

In [100]:
df['is_subscriber'] = df['is_subscriber'].apply(lambda x: 0 if x == False else 1)

In [101]:
df.head()

customer_id   country_name   age     customer_name  is_subscriber  \
0            1  united_states  21.0        Kasen Todd              1   
1            2      singapore  31.0      Ensley Garza              0   
2            3  united_states  22.0     Lillian Carey              0   
3            4  united_states  21.0  Beau Christensen              1   
4            5      singapore  22.0    Ernesto Gibson              1   

    subscriber_type  num_streams  
0    aavail_premium           23  
1  aavail_unlimited           12  
2    aavail_premium           22  
3      aavail_basic           19  
4    aavail_premium           23

In [102]:
## pull out the target and remove uneeded columns
_y = df.pop('is_subscriber')

In [103]:
y = np.zeros(_y.size)
y[_y==0] = 1

In [104]:
df.drop(columns=['customer_id', 'customer_name'], inplace=True)

In [105]:
df.head()

country_name   age   subscriber_type  num_streams
0  united_states  21.0    aavail_premium           23
1      singapore  31.0  aavail_unlimited           12
2  united_states  22.0    aavail_premium           22
3  united_states  21.0      aavail_basic           19
4      singapore  22.0    aavail_premium           23

In [52]:
# y==_y

First, we're going to remove the target from the data frame. So we're going to use the pop function to pop that out, and the target is is subscriber of course. After we do that, we switch churn to be the minority class. W

In [92]:
X_train , X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y)
print(sorted(Counter(y_train).items()))
print(sorted(Counter(y_test).items()))

[(0.0, 579), (1.0, 221)]
[(0.0, 145), (1.0, 55)]


In [97]:
df

country_name   age   subscriber_type  num_streams
0    united_states  21.0    aavail_premium           23
1        singapore  31.0  aavail_unlimited           12
2    united_states  22.0    aavail_premium           22
3    united_states  21.0      aavail_basic           19
4        singapore  22.0    aavail_premium           23
..             ...   ...               ...          ...
995      singapore  55.0  aavail_unlimited           14
996  united_states  23.0      aavail_basic           24
997  united_states  25.0  aavail_unlimited           17
998  united_states  40.0  aavail_unlimited           16
999  united_states  21.0  aavail_unlimited           18

[1000 rows x 4 columns]

# Transformation pipelines

In [74]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [114]:
numeric_features = ['age', 'num_streams']
numeric_transformers = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['country_name', 'subscriber_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformers, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegression(solver='lbfgs'))])

In [115]:
# import numpy as np
# from sklearn.impute import SimpleImputer
# imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant')
# imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])
# SimpleImputer()
# X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
# print(imp_mean.transform(X))


In [116]:
y_train, X_train

(array([1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 

In [118]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['subscriber', 'churn']))

              precision    recall  f1-score   support

  subscriber       0.84      0.94      0.89       145
       churn       0.76      0.53      0.62        55

    accuracy                           0.82       200
   macro avg       0.80      0.73      0.75       200
weighted avg       0.82      0.82      0.81       200



# Imbalanced learn
 * Simple interface and integrates with scikit-learn
 * Neural Network: imblearn.keras and imlearn.tensorflow
 * SMOTE and related variants
 * Works naturally with pipelines

In [121]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [120]:
!pip install imblearn

In [122]:
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2, 
                          n_redundant=0, n_repeated=0, n_classes=3,
                          n_clusters_per_class=1,
                          weights = [0.01, 0.05, 0.94],
                          class_sep=0.8, random_state=0)
print('Origignal Target')
print(sorted(Counter(y).items()))

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
print('\nResampled Target')
print(sorted(Counter(y_resampled).items()))

Origignal Target
[(0, 64), (1, 262), (2, 4674)]

Resampled Target
[(0, 4674), (1, 4674), (2, 4674)]


In [128]:
import imblearn.pipeline as pl
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# original pipeline
clf1 = pl.Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression(solver='lbfgs'))])
clf2 = pl.Pipeline(steps=[('preprocessor', preprocessor),
                         ('smote', RandomOverSampler(random_state=42)),
                         ('classifier', LogisticRegression(solver='lbfgs'))])
clf3 = pl.Pipeline(steps=[('preprocessor', preprocessor),
                         ('smote', SMOTE(random_state=42)),
                         ('classifier', LogisticRegression(solver='lbfgs'))])
for clf in [clf1, clf2, clf3]:
    clf.fit(X_train, y_train)

In [129]:
# !pip install cccc

In [130]:
for name, clf in [('no sampling', clf1), ('random oversamplint', clf), ('SMOTE',clf3)]:
    y_pred = clf.predict(X_test)
    print(name)
    print(classification_report(y_test, y_pred, target_names=['subscriber', 'churn']))

no sampling
              precision    recall  f1-score   support

  subscriber       0.84      0.94      0.89       145
       churn       0.76      0.53      0.62        55

    accuracy                           0.82       200
   macro avg       0.80      0.73      0.75       200
weighted avg       0.82      0.82      0.81       200

random oversamplint
              precision    recall  f1-score   support

  subscriber       0.87      0.84      0.85       145
       churn       0.61      0.65      0.63        55

    accuracy                           0.79       200
   macro avg       0.74      0.75      0.74       200
weighted avg       0.80      0.79      0.79       200

SMOTE
              precision    recall  f1-score   support

  subscriber       0.87      0.84      0.85       145
       churn       0.61      0.65      0.63        55

    accuracy                           0.79       200
   macro avg       0.74      0.75      0.74       200
weighted avg       0.80      0.79   

In [133]:
from sklearn.svm import SVC
clf_3 = SVC(kernel='linear',
         class_weight='balanced',
         probability=True)